# SOCOM GAP - Wolf Data Analysis

Dwell Time

In [1]:
import pandas as pd 
import os
import numpy as np

In [2]:
srcdir = os.getcwd()
dataFolder = srcdir + '/data/'
#outputFolder = srcdir + '/output/'

In [3]:
os.chdir(dataFolder)

In [4]:
files = os.listdir()
files

['AboVE-Wolf-event-data.csv',
 '.DS_Store',
 'cleaned.csv',
 'ABoVE-Wolf-reference-data.csv',
 'Wolf Telemetry Profile.html',
 'cleaned.xlsx']

In [5]:
wolfDf = pd.DataFrame()
f = 'AboVE-Wolf-event-data.csv'
print (f)

AboVE-Wolf-event-data.csv


In [6]:
#wolfDf = pd.read_csv(f, dtype={'event-id':'int', 'visible': 'boolean', 'tag-local-identifier':'string'})
wolfDf = pd.read_csv(f)

/Users/davidvennergrund/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
wolfDf

,event-id,visible,timestamp,location-long,location-lat,external-temperature,gps:dop,height-raw,sensor-type,individual-taxon-canonical-name,tag-local-identifier,individual-local-identifier,study-name,utm-easting,utm-northing,utm-zone,study-timezone,study-local-timestamp
0,9710294924,True,2013-12-19 00:00:44.000,-111.990073,57.212889,-22.0,2.6,350.59,gps,Canis lupus,13791,13791,ABoVE: Boutin Alberta Grey Wolf,440200.250982,6.341519e+06,12N,Mountain Standard Time,2013-12-18 17:00:44.000
1,9710294925,True,2013-12-19 12:00:45.000,-112.043078,57.211016,-8.0,1.6,335.05,gps,Canis lupus,13791,13791,ABoVE: Boutin Alberta Grey Wolf,436995.735523,6.341358e+06,12N,Mountain Standard Time,2013-12-19 05:00:45.000
2,9710294926,True,2013-12-20 00:00:44.000,-112.105655,57.260090,-5.0,1.8,409.08,gps,Canis lupus,13791,13791,ABoVE: Boutin Alberta Grey Wolf,433304.783325,6.346880e+06,12N,Mountain Standard Time,2013-12-19 17:00:44.000
3,9710294927,True,2013-12-20 12:00:44.000,-112.105336,57.261037,-12.0,1.6,414.98,gps,Canis lupus,13791,13791,ABoVE: Boutin Alberta Grey Wolf,433325.735759,6.346985e+06,12N,Mountain Standard Time,2013-12-20 05:00:44.000
4,9710294928,True,2013-12-21 00:00:44.000,-112.105630,57.260091,-9.0,1.8,416.13,gps,Canis lupus,13791,13791,ABoVE: Boutin Alberta Grey Wolf,433306.293102,6.346880e+06,12N,Mountain Standard Time,2013-12-20 17:00:44.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239189,9710389526,True,2014-06-20 19:00:29.000,-111.519134,56.720067,20.0,3.8,353.46,gps,Canis lupus,32263,32263B,ABoVE: Boutin Alberta Grey Wolf,468227.479806,6.286346e+06,12N,Mountain Daylight Time,2014-06-20 13:00:29.000
239190,9710389527,True,2014-06-21 07:00:53.000,-111.517532,56.721867,18.0,2.2,357.11,gps,Canis lupus,32263,32263B,ABoVE: Boutin Alberta Grey Wolf,468327.038830,6.286546e+06,12N,Mountain Daylight Time,2014-06-21 01:00:53.000
239191,9710389528,True,2014-06-21 19:00:29.000,-111.514991,56.722486,22.0,3.8,346.52,gps,Canis lupus,32263,32263B,ABoVE: Boutin Alberta Grey Wolf,468483.064131,6.286613e+06,12N,Mountain Daylight Time,2014-06-21 13:00:29.000
239192,9710389529,True,2014-06-22 07:00:53.000,-111.526562,56.713383,17.0,1.8,361.25,gps,Canis lupus,32263,32263B,ABoVE: Boutin Alberta Grey Wolf,467767.153505,6.285606e+06,12N,Mountain Daylight Time,2014-06-22 01:00:53.000


In [8]:
# drop constant columns, they never vary
drop = ['visible', 'individual-taxon-canonical-name', 'study-name', 'sensor-type', 
        'external-temperature', 'gps:dop','tag-local-identifier', 'study-local-timestamp', 'study-timezone']
df = wolfDf.drop(columns=drop)

## Time Difference

In [9]:
df['date'] = pd.to_datetime(df['timestamp'])

In [10]:
#sort by individual, then date
df = df.sort_values(by = ['individual-local-identifier', 'date'], ascending = [True, True], 
                    na_position = 'first')

In [11]:
# calculate time difference of date between rows (by individual-local-identifier)
df[['time diff']]=df.groupby('individual-local-identifier')[['date']].diff()

In [12]:
def elapsed (row):
    return row['time diff'].total_seconds()/3600

In [13]:
df[['elapsed hours']]=df.apply(lambda row: elapsed(row), axis=1)

## Distance

In [14]:
def dist (row):
    return ((row['utm-easting diff']**2) + (row['utm-northing diff']**2) + (row['height-raw diff']**2))**0.5

In [15]:
df[['height-raw diff']]=df.groupby('individual-local-identifier')[['height-raw']].diff()

In [16]:
df[['utm-easting diff','utm-northing diff']]=df.groupby('individual-local-identifier')[['utm-easting','utm-northing']].diff()

In [17]:
df[['distance']]=df.apply(lambda row: dist(row), axis=1)

## Speed

Distance Since Last Measurement / Time Elaspsed

12,000 meters / 12 hour = 1000 m/h
12 meters / 12 hours = 1 m/h

Small distances imply dwelling or hideout
Small speeds imply same plus ??

In [18]:
def speed (row):
    return (row['distance'] / row['elapsed hours'])

In [19]:
df[['speed m/hr']]=df.apply(lambda row: speed(row), axis=1)

In [20]:
drop = ['date']
df = df.drop(columns=drop)

In [21]:
file_name = 'cleaned.csv'
df.to_csv(file_name, encoding='utf-8', index=False)

In [22]:
df.head(10)

,event-id,timestamp,location-long,location-lat,height-raw,individual-local-identifier,utm-easting,utm-northing,utm-zone,time diff,elapsed hours,height-raw diff,utm-easting diff,utm-northing diff,distance,speed m/hr
661,9710295585,2013-12-20 00:00:43.000,-112.142245,57.049629,381.17,13790,430705.307449,6.323490e+06,12N,NaT,NaN,NaN,NaN,NaN,NaN,NaN
662,9710295586,2013-12-20 12:00:43.000,-111.964311,57.090529,378.62,13790,441563.599341,6.327876e+06,12N,0 days 12:00:00,12.000000,-2.55,10858.291892,4386.057287,11710.679222,975.889935
663,9710295587,2013-12-21 00:00:43.000,-111.947107,57.141068,319.19,13790,442684.205744,6.333487e+06,12N,0 days 12:00:00,12.000000,-59.43,1120.606404,5610.954965,5722.071850,476.839321
664,9710295588,2013-12-21 12:00:44.000,-111.947470,57.139597,320.26,13790,442659.964346,6.333324e+06,12N,0 days 12:00:01,12.000278,1.07,-24.241398,-163.434245,165.225733,13.768492
665,9710295589,2013-12-22 00:00:43.000,-111.967698,57.174972,331.42,13790,441491.711521,6.337278e+06,12N,0 days 11:59:59,11.999722,11.16,-1168.252825,3954.835394,4123.792212,343.657306
666,9710295590,2013-12-22 12:00:44.000,-111.947506,57.139597,329.80,13790,442657.785740,6.333324e+06,12N,0 days 12:00:01,12.000278,-1.62,1166.074219,-3954.805129,4123.131736,343.586358
667,9710295591,2013-12-23 00:01:02.000,-111.930870,57.058755,404.15,13790,443541.784308,6.324311e+06,12N,0 days 12:00:18,12.005000,74.35,883.998568,-9012.474715,9056.030139,754.354864
668,9710295592,2013-12-23 12:01:13.000,-112.228847,56.992627,411.55,13790,425337.716127,6.317237e+06,12N,0 days 12:00:11,12.003056,7.40,-18204.068180,-7074.528170,19530.414791,1627.120253
669,9710295593,2013-12-24 00:00:43.000,-112.092086,57.052683,389.54,13790,433753.504815,6.323780e+06,12N,0 days 11:59:30,11.991667,-22.01,8415.788688,6543.537982,10660.388032,888.983019
670,9710295594,2013-12-24 12:00:43.000,-112.073952,57.071599,371.12,13790,434886.595424,6.325868e+06,12N,0 days 12:00:00,12.000000,-18.42,1133.090609,2088.060889,2375.759226,197.979935
